# Kaggle project

Analyzing weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species

In [2]:
import numpy as np
import pandas as pd

In [5]:
# load datasets

weather = pd.read_csv('assets/weather.csv')
spray = pd.read_csv('assets/spray.csv')
train = pd.read_csv('assets/train.csv')
test = pd.read_csv('assets/test.csv')

In train and test data:
- NumMosquitos: number of mosquitoes caught in this trap
- WnvPresent: whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 


## Join weather and spray to a new training dataset

In [10]:
train.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [19]:
train.Date.nunique()

95

In [30]:
sl = zip(spray.Latitude, spray.Longitude)
rl = zip(train.Latitude, train.Longitude)

set(sl).intersection(rl)

set()

In [14]:
weather.head(5)

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [17]:
spray.Date.unique()

array(['2011-08-29', '2011-09-07', '2013-07-17', '2013-07-25',
       '2013-08-08', '2013-08-15', '2013-08-16', '2013-08-22',
       '2013-08-29', '2013-09-05'], dtype=object)

In [20]:
test.Date.unique()

array(['2008-06-11', '2008-06-17', '2008-06-24', '2008-07-01',
       '2008-07-04', '2008-07-11', '2008-07-14', '2008-07-21',
       '2008-07-23', '2008-07-24', '2008-07-28', '2008-07-29',
       '2008-08-04', '2008-08-05', '2008-08-12', '2008-08-13',
       '2008-08-19', '2008-08-25', '2008-08-26', '2008-09-02',
       '2008-09-03', '2008-09-09', '2008-09-15', '2008-09-19',
       '2008-09-29', '2010-06-02', '2010-06-11', '2010-06-18',
       '2010-06-25', '2010-06-28', '2010-07-01', '2010-07-02',
       '2010-07-12', '2010-07-13', '2010-07-16', '2010-07-19',
       '2010-07-23', '2010-07-26', '2010-07-29', '2010-07-30',
       '2010-08-05', '2010-08-06', '2010-08-13', '2010-08-19',
       '2010-08-20', '2010-08-26', '2010-08-27', '2010-09-02',
       '2010-09-10', '2010-09-13', '2010-09-16', '2010-09-17',
       '2010-09-23', '2010-09-24', '2010-10-01', '2012-06-08',
       '2012-06-15', '2012-06-21', '2012-06-29', '2012-07-09',
       '2012-07-13', '2012-07-19', '2012-07-20', '2012-

Drop spray data in train dataset

## EDA

## Build model

## Get result